In [2]:
import numpy as np, sys, os
sys.path.insert(0, os.path.join(os.path.abspath(os.getcwd()), "cmake-build-release"))
import pywrap
import random
import qgrid
from python import table_pb2
import pandas as pd


In [3]:
arr1 = np.repeat(np.arange(100), 10000).astype(np.uint32)
np.random.shuffle(arr1)
arr2 = np.repeat(np.arange(1000000), 1).astype(np.uint32)
np.random.shuffle(arr2)

In [4]:
%%timeit -n 10
v, inv = pywrap.fast_unique(arr1)
assert np.sum(v[inv] != arr1) == 0

7.74 ms ± 390 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [5]:
config = table_pb2.Table()
config.name = "securities_minute"
config.schema.time_column = "t"
config.schema.tag_column.append("s")
close_price = table_pb2.ColumnSchema()
close_price.name = "c"
close_price.type = table_pb2.ColumnSchema.FLOAT
config.schema.value_column.append(close_price)
table = pywrap.Table("/tmp/pydb/table_1", config.SerializeToString())

In [6]:
# for e in range(3, 1000):
#     table.append_data({
#         "t": np.arange(100000).astype(np.int64), 
#         "s": np.ones(100000).astype(np.uint32) * e, 
#         "c": np.arange(100000).astype(np.float32) / 10,
#     })
#     print(e)

In [7]:
sel = table_pb2.Selector()
sel.column.extend(["t", "s", "c"])
sel.time_selector.CopyFrom(table_pb2.TimeSelector(start=80000, end=80500, include_end=True))
# sel.sub_table_selector.tag_selector.append(table_pb2.TagSelector(name="s", value=["GOOG", "FB"]))
sel.sub_table_selector.tag_order.append("s")
result = table.query(sel.SerializeToString())
v, inv = pywrap.fast_unique(result["s"])
mapping = dict(zip(v, table.resolve_str_refs(v)))
result["s"] = np.array([mapping[e] for e in v], dtype=np.object)[inv]
df = pd.DataFrame(result, columns = sel.column)
print(len(df))
# qgrid.show_grid(df)


49599


In [8]:
%%timeit -n 100
sel = table_pb2.Selector()
sel.column.extend(["t", "s", "c"])
sel.time_selector.CopyFrom(table_pb2.TimeSelector(start=80000, end=80300, include_end=True))
sel.sub_table_selector.tag_selector.append(table_pb2.TagSelector(name="s", value=[str(e) for e in range(1, 500)]))
sel.sub_table_selector.tag_order.append("s")
result = table.query(sel.SerializeToString())
v, inv = pywrap.fast_unique(result["s"])
mapping = dict(zip(v, table.resolve_str_refs(v)))
result["s"] = np.array([mapping[e] for e in v], dtype=np.object)[inv]
pd.DataFrame(result)

9.52 ms ± 119 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
while 1:
    result = table.query(sel.SerializeToString())


In [10]:
sel.time_selector.

SyntaxError: invalid syntax (<ipython-input-10-bc454e085cbf>, line 1)